# Marco Teórico  
## ARIMA (log-returns) + Monte Carlo | Ventana 100 días

---

## 1. Series de Tiempo Financieras

En finanzas, los precios de activos suelen analizarse como **series de tiempo**, donde cada observación depende de valores pasados y factores aleatorios.  

Una forma estándar de transformar precios es el **retorno logarítmico**:

$$
r_t = \ln \left( \frac{P_t}{P_{t-1}} \right)
$$

donde:
- $P_t$ = precio de cierre en el tiempo $t$  
- $r_t$ = log-return en el tiempo $t$

Los log-returns son aditivos en el tiempo y facilitan el modelado estadístico.

---

## 2. Modelo ARIMA en Log-Returns

El modelo **ARIMA (Autoregressive Integrated Moving Average)** permite capturar:
- **AR (p):** dependencia de valores pasados.
- **I (d):** número de diferenciaciones aplicadas.
- **MA (q):** dependencia de errores pasados.

En este caso trabajamos con log-returns, que ya son **estacionarios**, por lo que $d=0$.

La representación general es:

$$
r_t = \phi_1 r_{t-1} + \phi_2 r_{t-2} + \dots + \phi_p r_{t-p} +
      \theta_1 \varepsilon_{t-1} + \theta_2 \varepsilon_{t-2} + \dots + \theta_q \varepsilon_{t-q} + \varepsilon_t
$$

donde:
- $\phi_i$ = coeficientes autoregresivos  
- $\theta_j$ = coeficientes de media móvil  
- $\varepsilon_t \sim \mathcal{N}(0, \sigma^2)$ = error blanco

El criterio de selección del mejor modelo se realiza con el **Akaike Information Criterion (AIC)**:

$$
AIC = -2 \ln(L) + 2k
$$

donde $L$ es la verosimilitud y $k$ el número de parámetros.  
Un **AIC menor** indica un mejor ajuste.

---

## 3. Reconstrucción del Precio

Una vez ajustado el ARIMA sobre los log-returns $\hat{r}_t$, los precios se reconstruyen mediante:

$$
\hat{P}_t = P_0 \cdot \exp \left( \sum_{i=1}^t \hat{r}_i \right)
$$

donde $P_0$ es el precio inicial de la ventana de análisis.

---

## 4. Simulación Monte Carlo

El método de **Monte Carlo** permite simular trayectorias posibles de precios futuros a partir de las distribuciones estimadas de retornos.

Cada trayectoria se genera como:

$$
P_{t+h}^{(j)} = P_t \cdot \exp \left( \sum_{k=1}^h r_{t+k}^{(j)} \right)
$$

donde:
- $h$ = horizonte de días futuros  
- $j$ = índice de la simulación  
- $r_{t+k}^{(j)}$ = retorno simulado en el día $t+k$

Al repetir este proceso $N$ veces, se construye una **distribución de escenarios futuros**.

---

## 5. Cuantiles y Bandas de Confianza

Con los resultados de Monte Carlo se pueden calcular intervalos de confianza para el precio futuro mediante cuantiles:

- Banda 95%: $[Q_{0.05}, Q_{0.95}]$  
- Banda 90%: $[Q_{0.10}, Q_{0.90}]$  
- Mediana: $Q_{0.50}$

Estos intervalos muestran el rango probable de evolución del activo, facilitando la toma de decisiones bajo incertidumbre.

---

## 6. Aplicación Práctica

1. **Ventana de 100 días:** garantiza que el modelo capture la dinámica más reciente.  
2. **ARIMA en log-returns:** asegura estacionariedad y reduce problemas de heterocedasticidad.  
3. **Monte Carlo con 500 simulaciones:** genera rutas futuras realistas considerando la aleatoriedad del mercado.  
4. **Gráficos y bandas:** permiten interpretar escenarios de riesgo y posibles precios futuros.

---

## 7. Conclusión

El enfoque **ARIMA + Monte Carlo**:
- Modela la dependencia temporal en los retornos.
- Simula precios futuros bajo incertidumbre.
- Genera intervalos de confianza robustos.
- Proporciona una herramienta práctica para **pronósticos financieros** y **gestión de riesgo**.

---

In [1]:
# ============================================================
# ARIMA (log-returns) + Monte Carlo | ventana 100 días (FIX numpy)
# ============================================================

# !pip install yfinance statsmodels plotly --quiet

import numpy as np
import pandas as pd
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.statespace.sarimax import SARIMAX
import plotly.graph_objects as go
from pandas.tseries.offsets import BDay

# -----------------------------
# Parámetros
# -----------------------------
TICKER      = "MELI" #"PLTR"      # Ej: "AAPL", "NVDA", "BTC-USD", "EURUSD=X"
PERIOD      = "4y"
FUTURE_DAYS = 100
MC_SIMS     = 1000 # simulaciones
TREND       = "n" #c Arima C
#trend_candidates = ["n", "c", "t", "ct", "d"]  # 'd' = drift (alias)
P_GRID      = [0,1,2,3,4,5] # posibles valores de p
D_GRID      = [0,1,2,3,4,5] # posibles valores de d
Q_GRID      = [0,1,2,3,4,5] # posibles valores de q

# -----------------------------
# 1) Descarga y limpieza
# -----------------------------
df = yf.download(TICKER, period=PERIOD, auto_adjust=True, progress=False).copy()

# Aplanar columnas si vienen en MultiIndex
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

# Si tu pipeline añadió 'Price', la quitamos para evitar confusiones
for extra_col in ["Price"]:
    if extra_col in df.columns:
        df.drop(columns=[extra_col], inplace=True)

needed = {"Open","High","Low","Close"}
missing = needed - set(df.columns)
if missing:
    raise ValueError(f"Faltan columnas necesarias: {missing}")

if df.empty or len(df) < 120:
    raise ValueError(f"No hay datos suficientes para {TICKER} en {PERIOD}.")

# Ventana 100 días
window_df = df.tail(100).copy()
window_df.index = pd.DatetimeIndex(window_df.index).tz_localize(None)
window_df.index.name = None

# -----------------------------
# 2) Log-returns
# -----------------------------
window_df["LogPrice"] = np.log(window_df["Close"])
r = window_df["LogPrice"].diff().dropna()
r.index = pd.DatetimeIndex(r.index).tz_localize(None)

# -----------------------------
# 3) Selección ARIMA por AIC
# -----------------------------
best_aic = np.inf
best_order, best_res = None, None

for p in P_GRID:
    for d in D_GRID:
        for q in Q_GRID:
            try:
                model = SARIMAX(
                    r, order=(p,d,q), trend=TREND,
                    enforce_stationarity=True, enforce_invertibility=True
                )
                res = model.fit(disp=False)
                if res.aic < best_aic:
                    best_aic = res.aic
                    best_order = (p,d,q)
                    best_res = res
            except Exception:
                continue

if best_res is None:
    raise RuntimeError("No se pudo ajustar un ARIMA con la grilla dada.")

print(f"✅ Mejor ARIMA en log-returns: ARIMA{model.order} trend='{model.trend}' | AIC={best_aic:.2f}")

# -----------------------------
# 4) Fitted histórico y precio modelado (FORZANDO NUMPY)
# -----------------------------
# Tomamos los fitted values como np.array para evitar reindexados implícitos
fv = np.asarray(best_res.fittedvalues)  # shape = (len(r),)

# En caso extremo de diferencias de largo por efectos de inicio, alineamos por tamaño mínimo
n = min(len(fv), len(r))
fv = fv[-n:]
r_idx = r.index[-n:]

# Precio base = primer Close de la ventana (robusto)
P0 = float(window_df["Close"].iloc[0])

# Reconstrucción: P_t = P0 * exp(cumsum(fv))
price_modeled = P0 * np.exp(np.cumsum(fv))
price_model_series = pd.Series(price_modeled, index=r_idx, name="Precio_Modelado")

# -----------------------------
# 5) Monte Carlo en retornos log (FORZANDO NUMPY)
# -----------------------------
sim_R = best_res.simulate(nsimulations=FUTURE_DAYS, anchor='end', repetitions=MC_SIMS)
sim_R = np.asarray(sim_R)  # <- evita DataFrame/MultiIndex

# paths: P_T * exp(cumsum(retornos))
last_close = float(window_df["Close"].iloc[-1])
paths = np.exp(np.log(last_close) + np.cumsum(sim_R, axis=0))  # (FUTURE_DAYS, MC_SIMS)

# Cuantiles por día (en numpy no hay sorpresas)
q05 = np.percentile(paths, 5, axis=1)
q10 = np.percentile(paths, 10, axis=1)
q50 = np.percentile(paths, 50, axis=1)
q90 = np.percentile(paths, 90, axis=1)
q95 = np.percentile(paths, 95, axis=1)

# Fechas hábiles futuras
last_date = window_df.index[-1]
future_dates = pd.date_range(start=last_date + BDay(1), periods=FUTURE_DAYS, freq=BDay())
future_dates.name = None

forecast_df = pd.DataFrame({
    "Forecast": q50,
    "Lower_90": q10,
    "Upper_90": q90,
    "Lower_95": q05,
    "Upper_95": q95
}, index=future_dates)

# -----------------------------
# 6) Gráficos (usando add_trace explícito)
# -----------------------------
fig = go.Figure()

# Velas
fig.add_trace(go.Candlestick(
    x=window_df.index,
    open=window_df["Open"], high=window_df["High"],
    low=window_df["Low"], close=window_df["Close"],
    name="Velas (100d)"
))

# Precio modelado histórico
fig.add_trace(go.Scatter(
    x=price_model_series.index, y=price_model_series.values,
    mode="lines", name=f"ARIMA{best_order} (histórico)", line=dict(width=2)
))

# Mediana futura
fig.add_trace(go.Scatter(
    x=forecast_df.index, y=forecast_df["Forecast"],
    mode="lines", name="Mediana MC", line=dict(width=2)
))

# Banda 95%
fig.add_trace(go.Scatter(
    x=list(forecast_df.index) + list(forecast_df.index[::-1]),
    y=list(forecast_df["Upper_95"]) + list(forecast_df["Lower_95"][::-1]),
    fill='toself', name='Banda 95%', opacity=0.20, line=dict(width=0)
))

# Banda 90%
fig.add_trace(go.Scatter(
    x=list(forecast_df.index) + list(forecast_df.index[::-1]),
    y=list(forecast_df["Upper_90"]) + list(forecast_df["Lower_90"][::-1]),
    fill='toself', name='Banda 90%', opacity=0.25, line=dict(width=0)
))

# Algunas rutas (pocas)
for j in range(min(10, MC_SIMS)):
    fig.add_trace(go.Scatter(
        x=forecast_df.index, y=paths[:, j],
        mode="lines", name=f"Ruta MC {j+1}", opacity=0.35, line=dict(width=1)
    ))

# Línea vertical de inicio de pronóstico
y_top = float(max(window_df["High"].max(), forecast_df["Upper_95"].max()))
y_bot = float(min(window_df["Low"].min(), forecast_df["Lower_95"].min()))
fig.add_shape(type="line", x0=last_date, x1=last_date, y0=y_bot, y1=y_top, line=dict(dash="dash", width=1))
fig.add_annotation(x=last_date, y=y_top, text="Inicio pronóstico", showarrow=True, arrowhead=2, ax=40, ay=-40)

fig.update_layout(
    title=f"{TICKER} — ARIMA{best_order} en log-returns + Monte Carlo ({FUTURE_DAYS} días)",
    xaxis_title="Fecha", yaxis_title="Precio",
    xaxis_rangeslider_visible=False, hovermode="x unified", template="plotly_white"
)

fig.show()

# -----------------------------
# 7) Tabla final
# -----------------------------
print("\nTabla de pronóstico — cuantiles diarios (primeras 10 filas):\n")
print(forecast_df.head(10).round(4).to_string())
print("\n…\nÚltimas 10:\n")
print(forecast_df.tail(10).round(4).to_string())

# Exporta CSV
forecast_df.to_csv(f"{TICKER}_forecast_ARIMA_MC.csv", index_label="Date")
print(f"\nCSV exportado: {TICKER}_forecast_ARIMA_MC.csv")

C:\Users\egarcia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\egarcia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\egarcia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored wh

✅ Mejor ARIMA en log-returns: ARIMA(5, 5, 5) trend='n' | AIC=-435.78



Tabla de pronóstico — cuantiles diarios (primeras 10 filas):

             Forecast   Lower_90   Upper_90   Lower_95   Upper_95
2026-01-26  2137.9359  2069.2096  2214.3400  2048.5117  2235.0197
2026-01-27  2139.2495  2038.3032  2245.1937  2008.0820  2272.1200
2026-01-28  2136.3884  2019.8489  2268.8927  1989.2463  2309.4709
2026-01-29  2140.5303  2005.0637  2277.1589  1970.2906  2323.6976
2026-01-30  2129.9982  1984.4896  2303.1766  1941.7760  2353.9900
2026-02-02  2143.6415  1966.6473  2322.1665  1924.7213  2359.8357
2026-02-03  2140.3805  1964.7165  2342.6911  1905.7059  2399.5607
2026-02-04  2142.8949  1941.8204  2349.9668  1898.6559  2418.0369
2026-02-05  2149.2908  1934.0314  2361.5812  1883.7073  2419.1176
2026-02-06  2146.9099  1933.6642  2376.6033  1869.6447  2433.5224

…
Últimas 10:

             Forecast   Lower_90   Upper_90   Lower_95   Upper_95
2026-06-01  2159.6904  1535.1078  2956.8858  1380.3298  3223.6393
2026-06-02  2144.3973  1523.7551  2960.0133  1386.6825  3200.68

In [2]:
res.fittedvalues

2025-09-03    0.000000
2025-09-04   -0.008382
2025-09-05    0.075063
2025-09-08   -0.112790
2025-09-09   -0.015307
                ...   
2026-01-16    0.004890
2026-01-20   -0.003241
2026-01-21   -0.007844
2026-01-22   -0.005067
2026-01-23    0.030887
Length: 99, dtype: float64

In [3]:
res.model.order

(5, 5, 5)